In [28]:
import pandas as pd
import numpy as np
domain_df = pd.read_csv("../../data/initial_data.csv")
# domain_df['new_domain'] = 'www.'+ domain_df.domain
domain_df.head(10)

,domain,domain_type
0,wisuolycossttqrj.com,dga
1,wi-wamss.org,benign
2,qcxfurnkbqidxxcl.biz,dga
3,192-168-1-1-admin.ru,benign
4,dblsiobnkjxomkmh.ru,dga
5,su5aksay.ru,benign
6,sisalex.edu.eg,benign
7,vloiolycossttqrj.cn,dga
8,bmabazar.com,benign
9,msyxwioqtcpbbypd.info,dga


In [29]:
expanded_df = pd.read_csv("../../data/expanded_data.csv")
expanded_df.head(10)

,domain,domain_type
0,nomadeducation.fr,benign
1,vsjmdqpmb.com,dga
2,nr1-logo-design-inspiration.tumblr.com,benign
3,180oqib10gr8s418aumdi12sl0u1.biz,dga
4,s50lan1mjcaed102lrxefl4356.com,dga
5,angelos-supplies.com,benign
6,egov.kz,benign
7,zfgcardenslavetusul.com,dga
8,lbyzestnessbiophysicalohax.com,dga
9,optionsplaybook.com,benign


In [30]:
df = pd.concat([domain_df,expanded_df])
df.drop_duplicates(inplace=True)
df.head()

,domain,domain_type
0,wisuolycossttqrj.com,dga
1,wi-wamss.org,benign
2,qcxfurnkbqidxxcl.biz,dga
3,192-168-1-1-admin.ru,benign
4,dblsiobnkjxomkmh.ru,dga


In [31]:
df.shape

(79287, 2)

In [33]:
# Creating feature
#Transforming each character from the domain to vowels.

def transform(val):
    vowel = ('a','e','i','o','u')
    new = ''
    for char in val:
        if char in vowel:
            new += 'v'
        elif char.isalpha() and char not in vowel:
            new += 'c'
        elif char.isdigit():
            new += 'n'
        else:
            new += 's'
    return new
df['abbreviation']= df['domain'].apply(transform)
domain_df.head(10)

,domain,domain_type
0,wisuolycossttqrj.com,dga
1,wi-wamss.org,benign
2,qcxfurnkbqidxxcl.biz,dga
3,192-168-1-1-admin.ru,benign
4,dblsiobnkjxomkmh.ru,dga
5,su5aksay.ru,benign
6,sisalex.edu.eg,benign
7,vloiolycossttqrj.cn,dga
8,bmabazar.com,benign
9,msyxwioqtcpbbypd.info,dga


In [34]:
from nltk.util import ngrams
def extract_ngrams(row, num):
    n_grams = ngrams(row, num)
    return [ ''.join(grams) for grams in n_grams]
for i in range(1,4):
    df['{}gram'.format(i)]= df['abbreviation'].apply(extract_ngrams, args=(i,))
df.head()

,domain,domain_type,abbreviation,1gram,2gram,3gram
0,wisuolycossttqrj.com,dga,cvcvvcccvcccccccscvc,"[c, v, c, v, v, c, c, c, v, c, c, c, c, c, c, ...","[cv, vc, cv, vv, vc, cc, cc, cv, vc, cc, cc, c...","[cvc, vcv, cvv, vvc, vcc, ccc, ccv, cvc, vcc, ..."
1,wi-wamss.org,benign,cvscvcccsvcc,"[c, v, s, c, v, c, c, c, s, v, c, c]","[cv, vs, sc, cv, vc, cc, cc, cs, sv, vc, cc]","[cvs, vsc, scv, cvc, vcc, ccc, ccs, csv, svc, ..."
2,qcxfurnkbqidxxcl.biz,dga,ccccvcccccvcccccscvc,"[c, c, c, c, v, c, c, c, c, c, v, c, c, c, c, ...","[cc, cc, cc, cv, vc, cc, cc, cc, cc, cv, vc, c...","[ccc, ccc, ccv, cvc, vcc, ccc, ccc, ccc, ccv, ..."
3,192-168-1-1-admin.ru,benign,nnnsnnnsnsnsvccvcscv,"[n, n, n, s, n, n, n, s, n, s, n, s, v, c, c, ...","[nn, nn, ns, sn, nn, nn, ns, sn, ns, sn, ns, s...","[nnn, nns, nsn, snn, nnn, nns, nsn, sns, nsn, ..."
4,dblsiobnkjxomkmh.ru,dga,ccccvvcccccvccccscv,"[c, c, c, c, v, v, c, c, c, c, c, v, c, c, c, ...","[cc, cc, cc, cv, vv, vc, cc, cc, cc, cc, cv, v...","[ccc, ccc, ccv, cvv, vvc, vcc, ccc, ccc, ccc, ..."


In [35]:
from collections import Counter
def generate_combinations(n, comb):
    if len(comb) == n:
        res.add(''.join(comb))
        return
    for char in ['c','v','s','n']:
        generate_combinations(n,comb+[char])
def bag_of_words_mapper(row, col):
    ngramlist = list(row)
    c = Counter(ngramlist)
    return c[col]
count = 1
for i in range(1,4):
    res = set()
    generate_combinations(i,[])
    for col in res:
        df[col] =  df['{}gram'.format(i)].apply(bag_of_words_mapper, args=(col,))
    df_ = df[list(res)]
    df['{}gram mean'.format(i)] = df_.mean(axis=1)
    df['{}gram variance'.format(i)] = df_.var(axis=1)
    df['{}gram std'.format(i)] = df_.std(axis=1)    

In [36]:
df['F15'] = df['domain'].apply(lambda x : len(set(x)))
df['F16'] = df['domain'].apply(lambda x : x.count('n')/len(x))
df['F17'] = df['domain'].apply(lambda x : x.count('c')/len(x))
df['F18'] = df['domain'].apply(lambda x : x.count('c')/x.count('v') if x.count('v') != 0 else x.count('c'))

In [38]:
X = df.drop(['domain','1gram','2gram','3gram','abbreviation'], axis=1)

In [39]:
y = X.pop('domain_type')

In [40]:
X.columns

Index(['n', 'c', 'v', 's', '1gram mean', '1gram variance', '1gram std', 'nv',
       'vn', 'ss', 'ns', 'sn', 'nc', 'sc', 'vv', 'cv', 'vs', 'nn', 'sv', 'vc',
       'cs', 'cn', 'cc', '2gram mean', '2gram variance', '2gram std', 'css',
       'cnv', 'cns', 'vnv', 'scv', 'ccc', 'nnc', 'nns', 'ncc', 'sns', 'nnn',
       'cnn', 'nss', 'nnv', 'snv', 'vss', 'csv', 'nvs', 'vvv', 'svn', 'ncn',
       'nvn', 'csn', 'cvv', 'vvn', 'svs', 'cvs', 'vns', 'snn', 'ccs', 'cnc',
       'vsv', 'vnc', 'nsc', 'nvv', 'scs', 'nsv', 'ncs', 'scn', 'vcc', 'ssv',
       'snc', 'sss', 'vnn', 'cvc', 'ccn', 'svc', 'svv', 'vvs', 'vvc', 'ssc',
       'nsn', 'cvn', 'vcv', 'csc', 'ssn', 'ccv', 'vcs', 'vcn', 'vsn', 'scc',
       'ncv', 'nvc', 'vsc', '3gram mean', '3gram variance', '3gram std', 'F15',
       'F16', 'F17', 'F18'],
      dtype='object')

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

In [24]:
from sklearn import datasets
datasets.dump_svmlight_file(X_train, y_train, "train.svmlight")
datasets.dump_svmlight_file(X_valid, y_valid, "valid.svmlight")

In [42]:
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(X_train, y_train)
y_predict =model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.85025
              precision    recall  f1-score   support

      benign       0.77      0.98      0.87      1978
         dga       0.98      0.72      0.83      2022

    accuracy                           0.85      4000
   macro avg       0.88      0.85      0.85      4000
weighted avg       0.88      0.85      0.85      4000



col_0,benign,dga
row_0,,
benign,1948,30
dga,569,1453


In [14]:
# save the model artifact
model_file_name = "locally-fixed-xgboost-model"
model._Booster.save_model(model_file_name)

In [15]:
# tar zip the model
!tar czvf explicit_feature_model.tar.gz $model_file_name

locally-fixed-xgboost-model


In [43]:
# tuning
model2=XGBClassifier(max_depth=5, learning_rate =0.07, random_state =575, n_estimators=200, n_jobs=4)
model2.fit(X_train, y_train)
y_predict =model2.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.846
              precision    recall  f1-score   support

      benign       0.77      0.98      0.86      1978
         dga       0.98      0.71      0.82      2022

    accuracy                           0.85      4000
   macro avg       0.87      0.85      0.84      4000
weighted avg       0.87      0.85      0.84      4000



col_0,benign,dga
row_0,,
benign,1946,32
dga,584,1438


In [48]:
# use Boruta
from boruta import BorutaPy
xgb=XGBClassifier(max_depth=2, learning_rate =0.07, random_state =575, n_estimators=200)
feat_selector = BorutaPy(xgb, n_estimators='auto', verbose=2, random_state=1)
feat_selector.fit(X_train.values, y_train.values)
feat_selector.support_
feat_selector.ranking_

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	9 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	10 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	11 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	12 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	13 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	14 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	15 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	16 / 100
Confirmed: 	23
Tentative: 	3
Reject

array([ 1, 40, 40,  1,  1,  1, 40, 40, 40, 40, 40,  5, 40, 40, 40,  1,  2,
       40, 40,  1, 40,  1,  1, 40,  1, 40, 40,  1, 40, 40, 40, 40, 40, 40,
        1,  1, 40, 40, 40, 40, 40,  1, 40, 40, 40, 40, 40,  4, 40, 40, 40,
        2, 40, 40, 40, 40, 40, 40, 40, 40, 40,  1,  3, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,  1, 40, 40,  1,  1,
        1, 40, 40, 40,  6, 40,  1, 40,  1,  1,  1,  1])

In [49]:
X_filtered_train = feat_selector.transform(X_train.values)
X_filtered_test = feat_selector.transform(X_test.values)

In [50]:
xgb_bo=XGBClassifier(max_depth=2, learning_rate =0.07, random_state =575, n_estimators=200)
xgb_bo.fit(X_filtered_train, y_train)
y_predict =xgb_bo.predict(X_filtered_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.9955
              precision    recall  f1-score   support

      benign       1.00      0.99      1.00      1978
         dga       0.99      1.00      1.00      2022

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



col_0,benign,dga
domain_type,,
benign,1960,18
dga,0,2022


In [51]:
X_filtered_test.shape
X_filtered_train.shape

(16000, 23)

In [52]:
X_train.columns

Index(['s', 'n', 'c', 'v', '1gram mean', '1gram variance', '1gram std', 'sn',
       'cs', 'nc', 'ns', 'sc', 'cn', 'vn', 'nv', 'cv', 'vc', 'sv', 'ss', 'cc',
       'nn', 'vv', 'vs', '2gram mean', '2gram variance', '2gram std', 'snn',
       'vcc', 'vcn', 'scs', 'svs', 'nnv', 'csv', 'nnc', 'vcv', 'ccs', 'svc',
       'css', 'cvn', 'nvn', 'ncs', 'vvc', 'csn', 'vvs', 'vnv', 'vsn', 'nsv',
       'ccv', 'ncn', 'vnc', 'vnn', 'svv', 'ssv', 'ssn', 'cnn', 'ncc', 'nvv',
       'nsn', 'sss', 'nvs', 'cnc', 'cvc', 'vcs', 'nsc', 'cnv', 'vvn', 'vsc',
       'csc', 'vns', 'svn', 'cns', 'nnn', 'nns', 'nvc', 'vss', 'ncv', 'vsv',
       'scn', 'ccn', 'snv', 'cvv', 'cvs', 'ssc', 'ccc', 'scv', 'scc', 'snc',
       'sns', 'nss', 'vvv', '3gram mean', '3gram variance', '3gram std', 'F15',
       'F16', 'F17', 'F18'],
      dtype='object')

In [55]:
res = set()
def generate_combinations(n, comb):
    if len(comb) == n:
        res.add(''.join(comb))
        return
    for char in ['c','v','s','n']:
        generate_combinations(n,comb+[char])
generate_combinations(2, [])
print(res)

{'sn', 'cs', 'nc', 'ns', 'sc', 'cn', 'vn', 'nv', 'cv', 'vc', 'sv', 'ss', 'cc', 'nn', 'vv', 'vs'}


In [31]:
assignment = pd.read_csv("assignment.csv")
assignment['feature']= assignment['domain'].apply(transform)
assignment

,domain,feature
0,www.dutchwebdesign.com,cccscvccccvccvcvccscvc
1,www.pploulsosohvmkv.com,cccscccvvccvcvcccccscvc
2,www.cfsjdqwdhmwkiv.com,cccsccccccccccccvcscvc
3,www.uocoqgiusyeciouaimcauykqswsymo.com,cccsvvcvccvvccvcvvvvvccvvccccccccvscvc
4,www.total-toolbar.com,cccscvcvcscvvccvcscvc
...,...,...
495,www.u23lib5bcxw0yju.com,cccsvnncvcnccccnccvscvc
496,www.prmysangh.com,cccscccccvcccscvc
497,www.ociqkeasqocgoecksu.com,cccsvcvccvvccvccvvcccvscvc
498,www.hj-9.com,cccsccsnscvc


In [44]:
real_time = pd.read_csv("../../data/feedback.csv",header=0,index_col=False)
real_time.head()

,domain,submitted,correct
0,cfsjdqwdhmwkiv,benign,dga
1,uocoqgiusyeciouaimcauykqswsymo,benign,dga
2,okdbnxoauhzrawyu,benign,dga
3,m8sdetc0u81lgdcpshoxsvy,benign,dga
4,qkdccn,benign,dga


In [45]:
real_time['abbreviation']= real_time['domain'].apply(transform)
real_time.head(10)

,domain,submitted,correct,abbreviation
0,cfsjdqwdhmwkiv,benign,dga,ccccccccccccvc
1,uocoqgiusyeciouaimcauykqswsymo,benign,dga,vvcvccvvccvcvvvvvccvvccccccccv
2,okdbnxoauhzrawyu,benign,dga,vcccccvvvcccvccv
3,m8sdetc0u81lgdcpshoxsvy,benign,dga,cnccvccnvnncccccccvcccc
4,qkdccn,benign,dga,cccccc
5,g2g0kfwnw65lqri,benign,dga,cncncccccnncccv
6,xretmysxptli,benign,dga,ccvccccccccv
7,cewesckwaqc,benign,dga,cvcvccccvcc
8,jknwld,benign,dga,cccccc
9,kuekgowceqaomwkyeyys,benign,dga,cvvccvccvcvvccccvccc


In [46]:
for i in range(1,4):
    real_time['{}gram'.format(i)]= real_time['abbreviation'].apply(extract_ngrams, args=(i,))
real_time.head()

,domain,submitted,correct,abbreviation,1gram,2gram,3gram
0,cfsjdqwdhmwkiv,benign,dga,ccccccccccccvc,"[c, c, c, c, c, c, c, c, c, c, c, c, v, c]","[cc, cc, cc, cc, cc, cc, cc, cc, cc, cc, cc, c...","[ccc, ccc, ccc, ccc, ccc, ccc, ccc, ccc, ccc, ..."
1,uocoqgiusyeciouaimcauykqswsymo,benign,dga,vvcvccvvccvcvvvvvccvvccccccccv,"[v, v, c, v, c, c, v, v, c, c, v, c, v, v, v, ...","[vv, vc, cv, vc, cc, cv, vv, vc, cc, cv, vc, c...","[vvc, vcv, cvc, vcc, ccv, cvv, vvc, vcc, ccv, ..."
2,okdbnxoauhzrawyu,benign,dga,vcccccvvvcccvccv,"[v, c, c, c, c, c, v, v, v, c, c, c, v, c, c, v]","[vc, cc, cc, cc, cc, cv, vv, vv, vc, cc, cc, c...","[vcc, ccc, ccc, ccc, ccv, cvv, vvv, vvc, vcc, ..."
3,m8sdetc0u81lgdcpshoxsvy,benign,dga,cnccvccnvnncccccccvcccc,"[c, n, c, c, v, c, c, n, v, n, n, c, c, c, c, ...","[cn, nc, cc, cv, vc, cc, cn, nv, vn, nn, nc, c...","[cnc, ncc, ccv, cvc, vcc, ccn, cnv, nvn, vnn, ..."
4,qkdccn,benign,dga,cccccc,"[c, c, c, c, c, c]","[cc, cc, cc, cc, cc]","[ccc, ccc, ccc, ccc]"


In [47]:
for i in range(1,4):
    res = set()
    generate_combinations(i,[])
    for col in res:
        real_time[col] =  real_time['{}gram'.format(i)].apply(bag_of_words_mapper, args=(col,))
    df_ = real_time[list(res)]
    real_time['{}gram mean'.format(i)] = df_.mean(axis=1)
    real_time['{}gram variance'.format(i)] = df_.var(axis=1)
    real_time['{}gram std'.format(i)] = df_.std(axis=1)    

In [48]:
real_time['F15'] = real_time['domain'].apply(lambda x : len(set(x)))
real_time['F16'] = real_time['domain'].apply(lambda x : x.count('n')/len(x))
real_time['F17'] = real_time['domain'].apply(lambda x : x.count('c')/len(x))
real_time['F18'] = real_time['domain'].apply(lambda x : x.count('c')/x.count('v') if x.count('v') != 0 else x.count('c'))

In [49]:
real_time.head()

,domain,submitted,correct,abbreviation,1gram,2gram,3gram,n,c,v,...,ncv,nvc,vsc,3gram mean,3gram variance,3gram std,F15,F16,F17,F18
0,cfsjdqwdhmwkiv,benign,dga,ccccccccccccvc,"[c, c, c, c, c, c, c, c, c, c, c, c, v, c]","[cc, cc, cc, cc, cc, cc, cc, cc, cc, cc, cc, c...","[ccc, ccc, ccc, ccc, ccc, ccc, ccc, ccc, ccc, ...",0,13,1,...,0,0,0,0.187500,1.583333,1.258306,12,0.000000,0.071429,1.0
1,uocoqgiusyeciouaimcauykqswsymo,benign,dga,vvcvccvvccvcvvvvvccvvccccccccv,"[v, v, c, v, c, c, v, v, c, c, v, c, v, v, v, ...","[vv, vc, cv, vc, cc, cv, vv, vc, cc, cv, vc, c...","[vvc, vcv, cvc, vcc, ccv, cvv, vvc, vcc, ccv, ...",0,16,14,...,0,0,0,0.437500,1.551587,1.245627,13,0.000000,0.100000,3.0
2,okdbnxoauhzrawyu,benign,dga,vcccccvvvcccvccv,"[v, c, c, c, c, c, v, v, v, c, c, c, v, c, c, v]","[vc, cc, cc, cc, cc, cv, vv, vv, vc, cc, cc, c...","[vcc, ccc, ccc, ccc, ccv, cvv, vvv, vvc, vcc, ...",0,10,6,...,0,0,0,0.218750,0.554563,0.744690,13,0.062500,0.000000,0.0
3,m8sdetc0u81lgdcpshoxsvy,benign,dga,cnccvccnvnncccccccvcccc,"[c, n, c, c, v, c, c, n, v, n, n, c, c, c, c, ...","[cn, nc, cc, cv, vc, cc, cn, nv, vn, nn, nc, c...","[cnc, ncc, ccv, cvc, vcc, ccn, cnv, nvn, vnn, ...",4,16,3,...,0,0,0,0.328125,1.017609,1.008766,18,0.000000,0.086957,2.0
4,qkdccn,benign,dga,cccccc,"[c, c, c, c, c, c]","[cc, cc, cc, cc, cc]","[ccc, ccc, ccc, ccc]",0,6,0,...,0,0,0,0.062500,0.250000,0.500000,5,0.166667,0.333333,2.0


In [51]:
real_X = real_time.drop(['domain','1gram','2gram','3gram','abbreviation',' submitted'], axis=1)

In [52]:
real_X.columns

Index([' correct', 'n', 'c', 'v', 's', '1gram mean', '1gram variance',
       '1gram std', 'nv', 'vn', 'ss', 'ns', 'sn', 'nc', 'sc', 'vv', 'cv', 'vs',
       'nn', 'sv', 'vc', 'cs', 'cn', 'cc', '2gram mean', '2gram variance',
       '2gram std', 'css', 'cnv', 'cns', 'vnv', 'scv', 'ccc', 'nnc', 'nns',
       'ncc', 'sns', 'nnn', 'cnn', 'nss', 'nnv', 'snv', 'vss', 'csv', 'nvs',
       'vvv', 'svn', 'ncn', 'nvn', 'csn', 'cvv', 'vvn', 'svs', 'cvs', 'vns',
       'snn', 'ccs', 'cnc', 'vsv', 'vnc', 'nsc', 'nvv', 'scs', 'nsv', 'ncs',
       'scn', 'vcc', 'ssv', 'snc', 'sss', 'vnn', 'cvc', 'ccn', 'svc', 'svv',
       'vvs', 'vvc', 'ssc', 'nsn', 'cvn', 'vcv', 'csc', 'ssn', 'ccv', 'vcs',
       'vcn', 'vsn', 'scc', 'ncv', 'nvc', 'vsc', '3gram mean',
       '3gram variance', '3gram std', 'F15', 'F16', 'F17', 'F18'],
      dtype='object')

In [54]:
X.columns

Index(['n', 'c', 'v', 's', '1gram mean', '1gram variance', '1gram std', 'nv',
       'vn', 'ss', 'ns', 'sn', 'nc', 'sc', 'vv', 'cv', 'vs', 'nn', 'sv', 'vc',
       'cs', 'cn', 'cc', '2gram mean', '2gram variance', '2gram std', 'css',
       'cnv', 'cns', 'vnv', 'scv', 'ccc', 'nnc', 'nns', 'ncc', 'sns', 'nnn',
       'cnn', 'nss', 'nnv', 'snv', 'vss', 'csv', 'nvs', 'vvv', 'svn', 'ncn',
       'nvn', 'csn', 'cvv', 'vvn', 'svs', 'cvs', 'vns', 'snn', 'ccs', 'cnc',
       'vsv', 'vnc', 'nsc', 'nvv', 'scs', 'nsv', 'ncs', 'scn', 'vcc', 'ssv',
       'snc', 'sss', 'vnn', 'cvc', 'ccn', 'svc', 'svv', 'vvs', 'vvc', 'ssc',
       'nsn', 'cvn', 'vcv', 'csc', 'ssn', 'ccv', 'vcs', 'vcn', 'vsn', 'scc',
       'ncv', 'nvc', 'vsc', '3gram mean', '3gram variance', '3gram std', 'F15',
       'F16', 'F17', 'F18'],
      dtype='object')

In [56]:
label = real_X.pop(' correct')

In [58]:
real_y =model.predict(real_X.values)

In [59]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(label.values,real_y))
print(classification_report(label.values,real_y))
pd.crosstab(label.values,real_y)

0.569620253164557
              precision    recall  f1-score   support

      benign       0.04      0.67      0.07         6
         dga       0.98      0.57      0.72       231

    accuracy                           0.57       237
   macro avg       0.51      0.62      0.40       237
weighted avg       0.96      0.57      0.70       237



col_0,benign,dga
row_0,,
benign,4,2
dga,100,131


In [32]:
real_data = pd.read_csv("../../data/assignment.csv",header=0,index_col=False)
real_data['threat'] = real_y

In [27]:
real_data.to_csv('../../data/fixed_assignment_prediction.csv', index=False)